In [1]:
import pandas as pd
import numpy as np
import datetime
import math
import os
import matplotlib.pyplot as plt
from pylab import rcParams

plt.style.use('ggplot')
%pylab inline --no-import-all

Populating the interactive namespace from numpy and matplotlib


In [2]:
pwd()

'/home/yuichiro/Miniconda/Weather/GribToCSV/Visualization'

In [4]:
year="2019"
month="04"
day=15
time="000000"
duration=7
def getfile(year,month,day,time):
    file_name = "Weather_data/PV_result_forecast"+str(year+month+day+time)+".csv"    
    return file_name

In [5]:
print(os.path.isfile( getfile(year,month,str(day),time) ))
#input_data1 = input_data1[['DateTime','wind_10m[m/s]', 'temperature','Tc(the PV cell temperature)[°C]','Ppv(the output of the PV array)[Wh]']].set_index(['DateTime'])

True


In [ ]:
def output_wind10(year,month,day,time,duration):
    input_data1 = pd.read_csv( getfile(year,month,str(day),time),encoding="SHIFT-JIS" )
    input_data1 = input_data1[['DateTime' ,'wind_10m[m/s]']].set_index(['DateTime'])
    name1 = "wind_10m_"+str(year+month)+str(day)+str(time)
    input_data1 = input_data1.rename(columns={'wind_10m[m/s]':name1})
    for i in range(1,duration):
        input_data2 = pd.read_csv( getfile(year,month,str(day+i),time),encoding="SHIFT-JIS" )
        input_data2 = input_data2[['DateTime', 'wind_10m[m/s]']].set_index(['DateTime'])
        name2 = "wind_10m_"+str(year+month)+str(day+i)+str(time)
        input_data2 = input_data2.rename(columns={'wind_10m[m/s]':name2})
        input_data1= pd.concat([input_data1,input_data2],axis=1,sort='False')
    return input_data1

output_wind10(year,month,day,time,duration)[:5]

In [ ]:
pd_wind = output_wind10(year,month,day,time,duration)


pd_wind.plot(subplots=True,figsize=(20,20))
plt.show()

In [ ]:
#PV_result_realtime20190415000000.csv
real_wind = pd.read_csv("Weather_data/PV_result_realtime20190415000000.csv" ,encoding="SHIFT-JIS" )
real_wind = real_wind.rename(columns={'Wind Speed(10m)[m/s]':'RealTime_wind'})
real_wind = real_wind.rename(columns={'Time':'DateTime'})
real_wind =real_wind[['DateTime', 'RealTime_wind']].set_index(['DateTime'])
real_wind[:3]

In [ ]:
pd_wind= pd.concat([real_wind,pd_wind],axis=1,sort='False')

In [ ]:
pd_wind.plot(figsize=(20,10),ylim=[0,5])

plt.show()

In [ ]:
pd_wind=pd_wind.reset_index()
pd_wind=pd_wind.rename(columns={'index':'Date'})

In [ ]:
pd_wind.columns.values

In [ ]:
output2 = pd_wind[[ 'wind_10m_20190415000000', 'wind_10m_20190416000000',\
       'wind_10m_20190417000000', 'wind_10m_20190418000000',\
       'wind_10m_20190419000000', 'wind_10m_20190420000000',\
       'wind_10m_20190421000000']]

#pd_temp2=pd_temp
pd_wind_original=pd_wind
output3=output2

pd_wind_original

In [ ]:
from matplotlib import cm, ticker
import matplotlib.dates as mdates
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

rcParams['figure.figsize'] = 40,20
plt.rcParams["font.size"] = 40


input1 = pd.to_datetime(pd_wind['Date'])
output1 = pd_wind['RealTime_wind']

fig = plt.figure()
fig.set_tight_layout(False)
ax1 = fig.add_subplot(1,1,1)
ax1.plot(input1, output1, color='Black',linestyle='solid', linewidth = 4, label='Realtime')
ax1.set_ylim([0, 10])

#output2
for column_name, item in output2.iteritems():
    ax1.plot(input1, item,linestyle='--', linewidth = 8, label=column_name)

ax1.set_title('Wind Speed(10m)[m/s]')
ax1.set_xlabel('date')
ax1.set_ylabel('Wind Speed(10m)[m/s]')
ax1.set_xticklabels(input1,rotation=0, size="small")

# 軸目盛の設定
ax1.xaxis.set_major_locator(mdates.HourLocator(interval=6))
ax1.xaxis.set_major_formatter(mdates.DateFormatter("%dth %H:%M"))

# 軸目盛ラベルの回転
labels = ax1.get_xticklabels()
plt.setp(labels, rotation=90);

# 凡例をまとめて出力する
fig.legend(loc="lower right",
           bbox_to_anchor=(1, 0.5, 0.4, .100), 
           borderaxespad=0.,
           ncol=1,
           mode="expand",
           title="LABEL NAME"
          ) 

ax1.grid(False) #目盛のごちゃごちゃ変更

In [ ]:
from ipywidgets import interact

def figplot(select_day,size1,size2,fontsize,grid,y_range):
    rcParams['figure.figsize'] = size1,size2
    plt.rcParams["font.size"] = fontsize
    input1 = pd.to_datetime(pd_wind['Date'])
    output1 = pd_wind['RealTime_wind']

    fig = plt.figure(figsize=(50,30),dpi=200)
    fig = plt.figure()
    fig.set_tight_layout(False)
    ax1 = fig.add_subplot(1,1,1)
    #ax1.patch.set_facecolor('white')
    ax1.plot(input1, output1, color='Black',linestyle='solid', linewidth = 6, label='Realtime')
    ax1.set_ylim([0, y_range])

    #output2
    for column_name, item in output2.iteritems():
        if column_name==select_day:
            ax1.plot(input1, item,linestyle='--', linewidth = 12, label=column_name)

    ax1.set_title('Wind Speed(10m)[m/s]')
    ax1.set_xlabel('date')
    ax1.set_ylabel('Wind Speed(10m)[m/s]')
    ax1.set_xticklabels(input1,rotation=0, size="small")

    # 軸目盛の設定
    ax1.xaxis.set_major_locator(mdates.HourLocator(interval=grid))
    ax1.xaxis.set_major_formatter(mdates.DateFormatter("%dth %H"))
    
    plt.grid(which='major', color='black',linestyle='--')

    # 軸目盛ラベルの回転
    labels = ax1.get_xticklabels()
    plt.setp(labels, rotation=90);

    # 凡例をまとめて出力する
    fig.legend(loc="lower right",
               bbox_to_anchor=(1, 0.5, 0.4, .100), 
               borderaxespad=0.,
               ncol=1,
               mode="expand",
               title="LABEL NAME"
              ) 

    ax1.grid(True) #目盛のごちゃごちゃ変更

In [ ]:
interact(figplot,select_day=[ 'wind_10m_20190415000000', 'wind_10m_20190416000000',\
       'wind_10m_20190417000000', 'wind_10m_20190418000000',\
       'wind_10m_20190419000000', 'wind_10m_20190420000000',\
       'wind_10m_20190421000000'],size1=(0,100, 1),size2=(0,50, 1),fontsize=(0,100,1),grid=(1,24,1),y_range=(1,25,1))

In [ ]:
#reference from https://pythondatascience.plavox.info/scikit-learn/%E5%9B%9E%E5%B8%B0%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E8%A9%95%E4%BE%A1
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,explained_variance_score

def figplot_dropna(select_day,size1,size2,fontsize,grid,y_range):
    rcParams['figure.figsize'] = size1,size2
    plt.rcParams["font.size"] = fontsize


    fig = plt.figure(figsize=(50,30),dpi=200)
    fig = plt.figure()
    fig.set_tight_layout(False)
    ax1 = fig.add_subplot(1,1,1)
    
    pd_wind2=pd_wind_original
    
    for column_name, item in output3.iteritems():
        if column_name==select_day:
            pd_wind3=pd.concat([pd_wind2['Date'],pd_wind2['RealTime_wind'],item],axis=1).dropna()
            ax1.plot(pd_wind3['Date'], pd_wind3[column_name],linestyle='--', linewidth = 12, label=column_name)
            pred_list=pd_wind3[column_name].tolist()
            
    #ax1.patch.set_facecolor('white')
    ax1.plot(pd_wind3['Date'], pd_wind3['RealTime_wind'], color='Black',linestyle='solid', linewidth = 6, label='Realtime')
    ax1.set_ylim([0, y_range])
    
    ax1.set_title('Wind Speed(10m)[m/s]')
    ax1.set_xlabel('date')
    ax1.set_ylabel('Wind Speed(10m)[m/s]')
    ax1.set_xticklabels(input1,rotation=0, size="small")

    # 軸目盛の設定
    ax1.xaxis.set_major_locator(ticker.MultipleLocator(grid))
    plt.grid(which='major', color='black',linestyle='--')

    # 軸目盛ラベルの回転
    labels = ax1.get_xticklabels()
    plt.setp(labels, rotation=45);

    # 凡例をまとめて出力する
    fig.legend(loc="lower right",
               bbox_to_anchor=(1, 0.5, 0.4, .100), 
               borderaxespad=0.,
               ncol=1,
               mode="expand",
               title="LABEL NAME"
              ) 

    ax1.grid(True) #目盛のごちゃごちゃ変更
    print( 'MAE, Mean Absolute Error :', mean_absolute_error(pd_wind3['RealTime_wind'].tolist(), pred_list ) )
    print( 'MSE, Mean Squared Error :', mean_squared_error(pd_wind3['RealTime_wind'].tolist(), pred_list ) )
    print( 'RMSE: Root Mean Squared Error :', np.sqrt(mean_squared_error(pd_wind3['RealTime_wind'].tolist(), pred_list )) )
    print( 'R2, R-squared, coefficient of determination :', r2_score(pd_wind3['RealTime_wind'].tolist(), pred_list ) )
    print( 'explained_variance_score :', explained_variance_score(pd_wind3['RealTime_wind'].tolist(), pred_list ) )
    
    
    explained_variance_score

In [ ]:
interact(figplot_dropna,select_day=[ 'wind_10m_20190415000000', 'wind_10m_20190416000000',\
       'wind_10m_20190417000000', 'wind_10m_20190418000000',\
       'wind_10m_20190419000000', 'wind_10m_20190420000000',\
       'wind_10m_20190421000000'],size1=(0,100, 1),size2=(0,50, 1),fontsize=(0,100,1),grid=(1,24,1),y_range=(1,25,1))